In [1]:
# Converts IP address into an integer (example: 192.168.1.10 = 192x256**3 + 168x256**2 + 1x256 + 10 = 3232235786)
# This makes it easier: avoids string operations, allows for counting
def ip_to_int(ip):
    A, B, C, D = map(int, ip.split('.'))
    return A*256**3 + B*256**2 + C*256 + D

# This does the inverse of the function above; turns the integer into a IP address
# Example: 3232235786  A = 3232235786 // 16777216 = 192  num = 3232235786 - (192 * 16777216) = 11010314 then continue for the rest, the answer will be 192.168.1.10
def int_to_ip(num):
    A = num // 256**3
    num %= 256**3
    B = num // 256**2
    num %= 256**2
    C = num // 256    
    D = num % 256
    return f"{A}.{B}.{C}.{D}"

# Converts CIDR into a subnet mask
# 2**32 = bits in whole IP address, so cidr represents the 1's or network bits in the IP address
def cidr_to_mask(cidr):
    mask_int = (2**32 - 1) - (2**(32 - cidr) - 1)
    return int_to_ip(mask_int), mask_int

# Finds the class for the ip address and the default mask of the IP address
def ip_to_mask(ip_int):
    A = ip_int // 256**3
    if 1 <= A <= 126:
        return "255.0.0.0"
    elif 128 <= A <= 191:
        return "255.255.0.0"
    elif 192 <= A <= 223:
        return "255.255.255.0"
    else:
        return "Invalid address"

#Function that calculates subnets 
def calculate_subnet(ip_cidr, required_hosts):
    import numpy as np

    # Separates IP and CIDR 
    ip_str, cidr_str = ip_cidr.split('/')
    cidr = int(cidr_str)
    ip_int = ip_to_int(ip_str)

   # Checks if IP address is valid; we don't typically use address that start with 127 (loopback), 224-239 (multicast) and 240-255 (experimental) 
    first_octet = int(ip_str.split('.')[0])
    if first_octet == 127 or first_octet > 223:
        print(f"IP address {ip_str} is reserved (loopback or multicast/experimental) and not typically used.")
        return None
        
    # Original mask
    orig_mask_str, orig_mask_int = cidr_to_mask(cidr)

    # Figuring out the host bits using a loop where it will stop when the host bits is greater than required hosts 
    host_bits = 0
    while (2**host_bits - 2) < required_hosts:
        host_bits += 1

    # Creates a new mask based on the host bits that was found prior
    new_cidr = 32 - host_bits
    new_mask_str, new_mask_int = cidr_to_mask(new_cidr)

    # Creating the first network where block size is the size of the network 
    network_int = ip_int - (ip_int % (2**host_bits))
    block_size = 2**host_bits

    # Calculating number of subnets 
    num_subnets = 2 ** (new_cidr - cidr)

    # Generates each subnet
    rows = []
    for i in range(num_subnets):
        net = network_int + i * block_size
        brd = net + block_size - 1
        rows.append([
            str(i + 1),  # Subnets start from 1
            int_to_ip(net),
            f"{int_to_ip(net+1)}-{int_to_ip(brd-1)}",
            int_to_ip(brd)
        ])

    # Puts subnets into a table for better viewing experience using numpy
    arr = np.array(rows)
    headers = np.array(["Subnet", "Network ID", "Valid Range", "Broadcast ID"])
    col_widths = [max(len(headers[i]), max(np.char.str_len(arr[:, i]))) for i in range(len(headers))]

    header_line = "  ".join(headers[i].ljust(col_widths[i]) for i in range(len(headers)))
    print(header_line)
    print("-" * len(header_line))
    for row in arr:
        print("  ".join(row[i].ljust(col_widths[i]) for i in range(len(row))))

    # Returns the values obtained 
    return {
        "IP_address": ip_str,
        "Given CIDR": cidr,
        "Given CIDR Mask": orig_mask_str,
        "New CIDR": new_cidr,
        "New Subnet Mask": new_mask_str,
        "Hosts per Subnet (max)": (2**host_bits)-2,
        "Number of Subnets Possible": num_subnets
    }

In [3]:
result = calculate_subnet("192.30.0.155/24", 100)
# Print summary info
for k, v in result.items():
    print(f"{k}: {v}")

Subnet  Network ID    Valid Range                Broadcast ID
-------------------------------------------------------------
1       192.30.0.128  192.30.0.129-192.30.0.254  192.30.0.255
2       192.30.1.0    192.30.1.1-192.30.1.126    192.30.1.127
IP_address: 192.30.0.155
Given CIDR: 24
Given CIDR Mask: 255.255.255.0
New CIDR: 25
New Subnet Mask: 255.255.255.128
Hosts per Subnet (max): 126
Number of Subnets Possible: 2


In [4]:
#testing the code for class A IP address 
result1 = calculate_subnet("1.15.10.155/8", 2000000)
# Print summary info
for k, v in result1.items():
    print(f"{k}: {v}")

Subnet  Network ID  Valid Range              Broadcast ID 
----------------------------------------------------------
1       1.0.0.0     1.0.0.1-1.31.255.254     1.31.255.255 
2       1.32.0.0    1.32.0.1-1.63.255.254    1.63.255.255 
3       1.64.0.0    1.64.0.1-1.95.255.254    1.95.255.255 
4       1.96.0.0    1.96.0.1-1.127.255.254   1.127.255.255
5       1.128.0.0   1.128.0.1-1.159.255.254  1.159.255.255
6       1.160.0.0   1.160.0.1-1.191.255.254  1.191.255.255
7       1.192.0.0   1.192.0.1-1.223.255.254  1.223.255.255
8       1.224.0.0   1.224.0.1-1.255.255.254  1.255.255.255
IP_address: 1.15.10.155
Given CIDR: 8
Given CIDR Mask: 255.0.0.0
New CIDR: 11
New Subnet Mask: 255.224.0.0
Hosts per Subnet (max): 2097150
Number of Subnets Possible: 8


In [5]:
#testing the code for class B IP address 
result2 = calculate_subnet("129.37.6.0/16", 12345)
# Print summary info
for k, v in result2.items():
    print(f"{k}: {v}")

Subnet  Network ID    Valid Range                  Broadcast ID  
-----------------------------------------------------------------
1       129.37.0.0    129.37.0.1-129.37.63.254     129.37.63.255 
2       129.37.64.0   129.37.64.1-129.37.127.254   129.37.127.255
3       129.37.128.0  129.37.128.1-129.37.191.254  129.37.191.255
4       129.37.192.0  129.37.192.1-129.37.255.254  129.37.255.255
IP_address: 129.37.6.0
Given CIDR: 16
Given CIDR Mask: 255.255.0.0
New CIDR: 18
New Subnet Mask: 255.255.192.0
Hosts per Subnet (max): 16382
Number of Subnets Possible: 4


In [6]:
#testing the code for class C IP address 
result3 = calculate_subnet("197.82.13.244/24", 20)
# Print summary info
for k, v in result3.items():
    print(f"{k}: {v}")

Subnet  Network ID     Valid Range                  Broadcast ID 
-----------------------------------------------------------------
1       197.82.13.224  197.82.13.225-197.82.13.254  197.82.13.255
2       197.82.14.0    197.82.14.1-197.82.14.30     197.82.14.31 
3       197.82.14.32   197.82.14.33-197.82.14.62    197.82.14.63 
4       197.82.14.64   197.82.14.65-197.82.14.94    197.82.14.95 
5       197.82.14.96   197.82.14.97-197.82.14.126   197.82.14.127
6       197.82.14.128  197.82.14.129-197.82.14.158  197.82.14.159
7       197.82.14.160  197.82.14.161-197.82.14.190  197.82.14.191
8       197.82.14.192  197.82.14.193-197.82.14.222  197.82.14.223
IP_address: 197.82.13.244
Given CIDR: 24
Given CIDR Mask: 255.255.255.0
New CIDR: 27
New Subnet Mask: 255.255.255.224
Hosts per Subnet (max): 30
Number of Subnets Possible: 8


In [7]:
#testing the code for loopback IP address 
result3 = calculate_subnet("127.82.13.244/16", 20000)

IP address 127.82.13.244 is reserved (loopback or multicast/experimental) and not typically used.


In [8]:
#testing the code for multicast IP address 
result4 = calculate_subnet("224.2.3.67/24", 20000)

IP address 224.2.3.67 is reserved (loopback or multicast/experimental) and not typically used.


In [9]:
#testing the code for experimental IP address 
result4 = calculate_subnet("241.0.91.82/24", 20000)

IP address 241.0.91.82 is reserved (loopback or multicast/experimental) and not typically used.
